In [2]:
import pandas as pd
import math

column_names = [
    "url",          # 0: https://...
    "title",        # 1: Vente Maison...
    "location",     # 2: Bray sur seine...
    "desc_short",   # 3: Description T4...
    "description",  # 4: À Vendre...
    "rooms",        # 5: 4
    "surface",      # 6: 92
    "price",        # 7: 139 750
    "agency",       # 8: L'ADRESSE CHESSY
    "agencement",   # 9: 3 chambres, 1 salle de bain
    "general",    # 10: (vide dans ton exemple)
    "annexes",      # 11: Cave
    "dependence",      # 12: Terrasse
    "id_tech",      # 13: 1764140340
    "reference"     # 14: ParuVendu...
]

# On charge tout en 'string' (dtype=str) pour éviter les erreurs de conversion au début
try:
    df = pd.read_csv(
        "data/results.csv",
        header=None,
        names=column_names,
        dtype=str,
        on_bad_lines='skip' # Si une ligne est mal formée, on la saute au lieu de planter
    )
except Exception as e:
    print(f"Erreur à l'ouverture du fichier : {e}")
    df = pd.DataFrame()

df.head()  # Affiche les premières lignes du DataFrame pour vérification²  

,url,title,location,desc_short,description,rooms,surface,price,agency,agencement,general,annexes,dependence,id_tech,reference
0,https://www.paruvendu.fr/immobilier/vente/mais...,Vente Maison 4 pièces 92 m²,Bray sur seine (77480),Description T4 à Bray sur seine,À Vendre - Maison avec fort potentiel sur 1 54...,4,92,139 750,L'ADRESSE CHESSY,"3 chambres,1 salle de bain",NaN,Cave,Terrasse,1764140340,ParuVendu 14156758_1283B
1,https://www.paruvendu.fr/immobilier/vente/mais...,Vente Maison 5 pièces 132 m²,Provins (77160),Description T5 à Provins,Propriété de charme - Deux maisons + dépendanc...,5,132,139 750,L'ADRESSE CHESSY,"3 chambres,2 salles d'eau",NaN,Cave,NaN,1764140340,ParuVendu 14156709_1284P
2,https://www.paruvendu.fr/immobilier/vente/mais...,Vente Maison 7 pièces 150 m²,Orsay (91400),Description T7 à Orsay,Maison familiale 7 pièce(s) 150 m2 L'équipe l'...,7,150,520 000,L'ADRESSE ORSAY,5 chambres,NaN,NaN,Terrasse,1764145200,ParuVendu 14106019_1495
3,https://www.paruvendu.fr/immobilier/vente/appa...,Vente Appartement 3 pièces 56 m²,Le mee-sur-seine (77350),Description T3 à Le mee-sur-seine,Appartement très lumineux Au Mée sur Seine dan...,3,56,143 000,MEGAGENCE Vanessa MOLET,2 chambres,Chauffage : collectif sol,Parking,Balcon,1763752440,ParuVendu 193090_8920
4,https://www.paruvendu.fr/immobilier/vente/mais...,Vente Maison 6 pièces 110 m²,La grande-paroisse (77130),Description T6 à La grande-paroisse,Maison individuelle À La Grande Paroisse belle...,6,110,275 000,MEGAGENCE Vanessa MOLET,4 chambres,Chauffage : individuel,NaN,Terrasse,1763752440,ParuVendu 193089_8920


In [3]:
# Traitement des données pour ne récuperer que les colonnes nécessaires ( Groupement des annonces par Ville avec la sommme des chambres )
grouped_df = df.groupby('location').agg({
    'rooms': 'sum',
    'url': 'count'
}).rename(columns={'url': 'total_listings'}).reset_index()

In [4]:
from geopy.geocoders import Nominatim
import pandas as pd
import re, time, math

geolocator = Nominatim(user_agent="scraping_app", timeout=10)

# Fonction pour enlever les codes postaux entre parenthèses
def remove_postal_code(location):
    if not location:
        return location
    return re.sub(r"\s*\(\d{5}\)", "", location).strip()

# Fonction de géocodage avec gestion des erreurs et délai
def geocode_location(location, i, total):
    try:
        formatted = remove_postal_code(location)
        print(f"[{i}/{total}] Géocodage : {formatted}")
        time.sleep(1)
        loc = geolocator.geocode(formatted,exactly_one=True)
        if loc:
            return loc.latitude, loc.longitude
    except Exception:
        pass
    return math.nan, math.nan

total = len(grouped_df)

grouped_df[['latitude', 'longitude']] = [
    geocode_location(loc, i, total)
    for i, loc in enumerate(grouped_df['location'], start=1)
]


[1/556] Géocodage : Ablon sur seine
[2/556] Géocodage : Acheres
[3/556] Géocodage : Adainville
[4/556] Géocodage : Alfortville
[5/556] Géocodage : Andilly
[6/556] Géocodage : Andrésy
[7/556] Géocodage : Angerville
[8/556] Géocodage : Antony
[9/556] Géocodage : Arcueil
[10/556] Géocodage : Argenteuil
[11/556] Géocodage : Arnouville
[12/556] Géocodage : Arnouville les gonesse
[13/556] Géocodage : Arpajon
[14/556] Géocodage : Arronville
[15/556] Géocodage : Asnieres sur seine
[16/556] Géocodage : Asnieres-sur-seine
[17/556] Géocodage : Asnières-sur-seine
[18/556] Géocodage : Athis mons
[19/556] Géocodage : Athis-mons
[20/556] Géocodage : Attainville
[21/556] Géocodage : Aubergenville
[22/556] Géocodage : Aubervilliers
[23/556] Géocodage : Auffreville brasseuil
[24/556] Géocodage : Aulnay sous bois
[25/556] Géocodage : Aulnay-sous-bois
[26/556] Géocodage : Auteuil
[27/556] Géocodage : Auvers-sur-oise
[28/556] Géocodage : Avon
[29/556] Géocodage : Avrainville
[30/556] Géocodage : Bagnolet
[

In [7]:
# Sauvegarde du DataFrame géocodé
try: 
    grouped_df.to_csv("data/results_geocoded.csv", index=False) 
    print("Données géocodées sauvegardées dans data/results_geocoded.csv") 
except Exception as e: print(f"Erreur à l'enregistrement du fichier : {e}") 

Données géocodées sauvegardées dans data/results_geocoded.csv


In [8]:
# Utilisation de Folium pour visualiser les données sur une carte
import folium
from IPython.display import display

try:
    df = pd.read_csv("data/results_geocoded.csv", dtype=str)
    # Initialiser la carte centrée sur la France
    map_center = [48.8499198, 2.6370411]  # Coordonnées approximatives du centre de la France
    folium_map = folium.Map(location=map_center, zoom_start=6)
    # Ajouter des marqueurs pour chaque ville
    for _, row in df.iterrows():
        if pd.notna(row['latitude']) and pd.notna(row['longitude']):
            folium.Marker(
                location=[float(row['latitude']), float(row['longitude'])],
                popup=f"{row['location']}: {row['total_listings']} annonces"
            ).add_to(folium_map)
    # Sauvegarder la carte dans un fichier HTML
    folium_map.save("data/map.html")
    display(folium_map)
    print("Carte sauvegardée dans data/map.html")
except Exception as e:
    print(f"Erreur lors de la création de la carte : {e}")

Carte sauvegardée dans data/map.html
